In [ ]:
from skimage.io import imread,imshow,imsave
from matplotlib import pyplot as plt
from skimage.transform import resize
import numpy as np

In [ ]:
def ds_pred(i,j,Mn,Mf,Mfn,frames):
    l=len(frames)
    n=Mn[0]*Mn[1]+Mn[0]*Mfn[1]+Mfn[0]*Mn[1]
    f=Mf[0]*Mf[1]+Mf[0]*Mfn[1]+Mfn[0]*Mf[1]
    fn=Mfn[0]*Mfn[1]
    if l>2:
        for k in range(2,l):
            n=n*(Mn[k]+Mfn[k])+fn*Mn[k]
            f=f*(Mf[k]+Mfn[k])+fn*Mf[k]
            fn*=Mfn[k]
    K=n+f+fn
    N=n/K
    F=f/K
    FN=fn/K
    BN=N
    BF=F
    BFN=F+N

    PN=BN+FN
    PF=BF+FN
    PFN=BFN
    order=0
    if PN>PF:  #云
        if (np.sum(Mn-Mf)>0)==Mn.shape[0]:
            result=Mn[np.argmin(Mn)]
            frame_value=frames[np.argmin(Mn)][i][j]
            order=np.argmin(Mn)
        else:
            result=min(1-0.6,1-Mf[np.argmax(Mf)])
            frame_value = frames[np.argmax(Mf)][i][j]
            order=np.argmax(Mf)
    else: 
        result=min(1-Mn[np.argmax(Mf)],1-0.6)
        frame_value=frames[np.argmax(Mf)][i][j]
        order=np.argmax(Mf)
    pixs=frames[0:frames.shape[0],i,j]
    return order,result,frame_value

In [ ]:
#loading cloudy images and segmentation results in frames[length:384:384:3] and preds[length:384:384:3],their piexel values should be scaled to [0,1].
# length=5
# Order=7
# pred_path='F:\local\cloud-net\predictions/'
# preds=[]
# for i in range(length):
#     pred=imread(pred_path+'area'+str(Order)+'-'+str(i)+'_pred.tiff')
#     preds.append(pred)
# len(preds)
# frame_path='F:\local\cloud-net\datas/'
# frames=[]
# for i in range(length):
#     frame=imread(frame_path+'area'+str(Order)+'-'+str(i)+'.tiff')
#     frame=frame[:,:,0:3]
#     frame= resize (frame, ( 384, 384), preserve_range=True, mode='symmetric')
#     frames.append(frame/65535)
# for i in range(length):
#     for j in range(frames[i].shape[0]):
#         for k in range(frames[i].shape[1]):
#             if frames[i][j][k][0]>white_thre and frames[i][j][k][1]>white_thre and frames[i][j][k][2]>white_thre:
#                 preds[i][j][k]=min(0.999,preds[i][j][k]+0.5)
# preds=np.array(preds)
# frames=np.array(frames)

In [ ]:
white_thre=0.3  #gray threshold to help classifying clouds
frame=np.zeros(shape=frames[0].shape)
Mfn=[]
for i in range(length):
    Mfn.append(0.1)
Mfn=np.array(Mfn)
img = preds[0].copy()
for i in range(len(img)):
    for j in range(len(img)):
        Mn=[]
        Mf=[]
        for k in range(len(frames)):
              Mpn = preds[k][i][j]*(1-Mfn[k])
              Mn.append(Mpn[0])
              Mpf = (1-preds[k][i][j])*(1-Mfn[k])
              Mf.append(Mpf[0])
        Mn=np.array(Mn)
        Mf=np.array(Mf)
        order,img[i][j],frame[i][j]= ds_pred(i,j,Mn,Mf,Mfn,frames)
# frame=exposure.adjust_gamma(frame,0.8)) #If the result image is dark, you may try gamma exposure.
plt.imshow(frame)